In [1]:
# Standard imports
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import xarray as xr
import pandas as pd
import joblib
import pickle

# Machine learning libraries
import sklearn            # machine-learning libary with many algorithms implemented
import xgboost as xgb     # extreme gradient boosting (XGB)
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Python file with supporting functions
import model_utils

2023-01-03 17:58:54.017100: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
X = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/X.nc') 
y = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/y.nc') 
X_train = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/X_train.nc') 
y_train = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/y_train.nc') 
X_test = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/X_test.nc') 
y_test = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/y_test.nc') 

In [ ]:
X_unseen = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/X_unseen.nc') 
y_unseen = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/split_datasets/y_unseen.nc') 

In [12]:
X_train.to_dataframe()

SSS       SST         MLD       Chl  \
xlon   ylat  time                                                    
-179.5 -77.5 1996-12-15  34.519566 -1.687043  214.616638  2.408046   
             1997-01-15  34.359451  0.538524   18.861521  7.900837   
             1998-01-15  34.372940 -0.129260   48.051937  8.482795   
             2000-02-15  34.287220  2.750365   19.898767  0.205808   
             2003-01-15  34.288605 -0.362210   22.430550  8.158799   
...                            ...       ...         ...       ...   
 179.5  78.5 1990-07-15        NaN       NaN         NaN       NaN   
             1990-12-15        NaN       NaN         NaN       NaN   
             1989-11-15        NaN       NaN         NaN       NaN   
             1989-12-15        NaN       NaN         NaN       NaN   
             1988-11-15        NaN       NaN         NaN       NaN   

                               XCO2        T0        T1         A         B  \
xlon   ylat  time                                                             
-179.5 -77.5 1996-12-15  362.937073  0.966848 -0.255353 -0.976296 -0.001889   
             1997-01-15  363.078552  0.966848  0.255353 -0.976296 -0.001889   
             1998-01-15  365.225403  0.966848  0.255353 -0.976296 -0.001889   
             2000-02-15  368.928192  0.702527  0.711657 -0.976296 -0.001889   
             2003-01-15  374.658783  0.966848  0.255353 -0.976296 -0.001889   
...                             ...       ...       ...       ...       ...   
 179.5  78.5 1990-07-15         NaN       NaN       NaN       NaN       NaN   
             1990-12-15         NaN       NaN       NaN       NaN       NaN   
             1989-11-15         NaN       NaN       NaN       NaN       NaN   
             1989-12-15         NaN       NaN       NaN       NaN       NaN   
             1988-11-15         NaN       NaN       NaN       NaN       NaN   

                                C  
xlon   ylat  time                  
-179.5 -77.5 1996-12-15  0.216431  
             1997-01-15  0.216431  
             1998-01-15  0.216431  
             2000-02-15  0.216431  
             2003-01-15  0.216431  
...                           ...  
 179.5  78.5 1990-07-15       NaN  
             1990-12-15       NaN  
             1989-11-15       NaN  
             1989-12-15       NaN  
             1988-11-15       NaN  

[17747280 rows x 10 columns]

In [13]:
y_test.to_dataframe()

pCO2
xlon   ylat  time                  
-179.5 -77.5 1997-02-15  166.861496
             2005-01-15  174.326938
             2005-11-15  313.570954
             2007-02-15  185.846402
             2015-01-15  173.178833
...                             ...
 179.5  73.5 1983-10-15         NaN
             1986-09-15         NaN
             1991-04-15         NaN
             1995-01-15         NaN
             1985-06-15         NaN

[4465080 rows x 1 columns]

## If do standardization:

*Note: Done here, as opposed to in test/train split, so that I can save the original train/test datasets and later determine whether normalization led to improvement. (My group has not historically normalized data before training.)*

In [ ]:
X_norm = (X - X.mean())/X.std() 
y_norm = (y - y.mean())/y.std()
X_train_norm = (X_train - X_train.mean())/X_train.std()
y_train_norm = (y_train - y_train.mean())/y_train.std()
X_test_norm = (X_test - X_test.mean())/X_test.std()
y_test_norm = (y_test - y_test.mean())/y_test.std()

## If do additional validation split

### Train/validate/test split proportions

In [ ]:
# 20% of dataset for validation, 20% of dataset for testing, the rest for training
# Training set will be split into validation and another training set

val_prop = .2
test_prop = .2

In [ ]:
Xn = X.to_numpy()         
yn = y.to_numpy().ravel()    

In [ ]:
Xtrain_n = Xtrain.to_numpy() # create Xtrain and Xtest to randomly select from for X_train and X_test
ytrain_n = ytrain.to_numpy().ravel()

In [ ]:
# We don't use X_test_tmp or y_test_tmp when test years are used (X_test and y_test set above)
X_test_n = X_test.to_numpy()  #  Test metrics on all of SOCAT data from test years
y_test_n = y_test.to_numpy().ravel()

In [ ]:
# After re-import all the datasets, 

In [ ]:
# Splitting the training data into validation and another training set   
#N = Xtrain.shape[0]
#train_val_idx, train_idx, val_idx, test_idx = proc_utils.train_val_test_split(N, test_prop, val_prop, random_seeds, seed_loc)
#X_train_val, X_train, X_val, X_test_tmp, y_train_val, y_train, y_val, y_test_tmp = proc_utils.apply_splits(Xtrain, ytrain, train_val_idx, train_idx, val_idx, test_idx) 

In [ ]:
# Splitting the training data into validation and another training set   
N = Xtrain.shape[0]
train_val_idx, train_idx, val_idx, test_idx = proc_utils.train_val_test_split(N, test_prop, val_prop, random_seeds, seed_loc)
X_train_val, X_train, X_val, X_test_tmp, y_train_val, y_train, y_val, y_test_tmp = proc_utils.apply_splits(Xtrain, ytrain, train_val_idx, train_idx, val_idx, test_idx) 

# eXtreme Gradient Boosting 

In [ ]:
# A, B and C represent lon and lat (3 components of the n-vector; so that the algorithm doesn't interpret 0 and 360
# degrees to be far apart 
# T0 and T1 represent time

#features_sel = ['SSS','SST','MLD','Chl','XCO2','T0', 'T1','A', 'B', 'C'] 
#target_sel = ['pCO2'] 

### Referencing Group Methods

#### If chose to track seeds:

In [15]:
ens = 'CESM'
member = '009'
reference_output_dir = '/home/julias/MLEE-final-project/pickle_files'

path_seeds = f'{reference_output_dir}/random_seeds.npy'
random_seeds = np.load(path_seeds)  

path_loc = f'{reference_output_dir}/cesm_seed_loc_dict.pickle'
with open(path_loc,'rb') as handle:
    seed_loc_dict = pickle.load(handle)
seed_loc = seed_loc_dict[ens][member]

# for next project, where train and test datasets use all CESM members
#path_cesm = f"{reference_output_dir}/cesm_members_dict.pickle"
#with open(path_cesm,'rb') as handle:
#    cesm_mems_dict = pickle.load(handle)

NameError: name 'ens' is not defined

In [32]:
# random_seeds

In [33]:
# seed_loc_dict # using CESM 009, so seed_loc should be 32

In [34]:
# seed_loc #confirmed 32

#### Reference Best Parameters from Previous Group Work

Published in Bennington 2022, trained XGB to learn pCO2 residual (pC02 change with direct temperature effects removed)

In [35]:
path_bp='/data/artemis/workspace/vbennington/full_sst/pCO2_DIC/models/performance_metrics/xg/xg_best_params_dict.pickle'
with open(path_bp,'rb') as handle:
    best_params = pickle.load(handle)
print(best_params)

{'CESM': {'max_depth': 6, 'n_estimators': 4000}, 'GFDL': {'max_depth': 6, 'n_estimators': 4000}, 'MPI': {'max_depth': 6, 'n_estimators': 4000}, 'CanESM2': {'max_depth': 6, 'n_estimators': 4000}}


Use previous CESM best parameters as a starting point:
- max_depth = 6
- n_estimators = 4000

In [36]:
# Test with three different n_estimators and 3 different depths
# Advice from group post-doc: For XGB, too many depth layers may lead to overfitting (usually 8 or less layers for XGB)
# We want the combo of xg_param_grid that gives the lowest RMSE

xg_param_grid = {'n_estimators':[3000, 4000, 5000],
                 'max_depth':[5, 6, 7]}

(to compare performance to other methods from group)

In [13]:
# name everything with XGB
# will be Notebook B in 3_code

## Building and Training the XGB Model

In [ ]:
# best_params = {}  # opened above
# defaultdict is from a package #which one, does what

test_performance = defaultdict(dict)
unseen_performance = defaultdict(dict)

In [ ]:
# =========================================
# Number of cores you have access to for model training
# =========================================
jobs = 30

### Create two dictionaries

### XGB-Specific Inputs

In [ ]:
# K_folds: cross validation; number of splits for training set (in this case 3 splits; see below)
# Train on the first split, test on the remaining 2. Total 3 numbers for the final RMSE

K_folds = 3       # Split training set into 3 parts
approach = "xg"   # XGB approach
first_mem = False # Initialize if using gridsearch to find best_params

In [ ]:
# model: define which approach to use
# param_grid: the n_estimators (decision trees) and depths
# GridSearchCV: applying the K-fold cross validation
# first_mem = False: checks only the best params for first_mem (and not all members). For the next members, the  
# parameters from first_mem are re-used
# you could try to find the best params for a few members (but not all of them)
# 9 possible combinations (3 different n_estimators, 3 different max depths) x 3 (K-fold; 3 training sets)
if first_mem:
            model = XGBRegressor(random_state=random_seeds[4,seed_loc], n_jobs=jobs)
            param_grid = xg_param_grid
            grid = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=K_folds, return_train_score=False, refit=False)
            grid.fit(X_train_val, y_train_val)
            best_params[ens] = grid.best_params_
            print(best_params)
            first_mem = False

### Train the XBG model

In [ ]:
model = XGBRegressor(random_state=random_seeds[5,seed_loc], **best_params[ens], n_jobs=jobs)
model.fit(X_train_val, y_train_val) 

### Save the XBG model

In [ ]:
#Uncomment when actually running

# pre_saildrone now called utils, carry through this change
utils.save_model(model, model_output_dir, approach, ens, member)
print(datetime.datetime.now())
print(ens)
print(member)

## Testing the XGB Model

### Preliminary Analysis on XBG Test Error Metrics

In [ ]:
# Calculate some test error metrics and store in a dictionary
# evaluate_test is a function from pre_saildrone. it includes MSE, MAE, bias etc

y_pred_test = model.predict(X_test)

test_performance[ens][member] = utils.evaluate_test(y_test, y_pred_test)
print(test_performance[ens][member])

In [ ]:
# Redo this analysis on the unseen data
y_pred_unseen = model.predict(df.loc[unseen_sel,features_sel].to_numpy())

y_unseen = df.loc[unseen_sel,target_sel].to_numpy().ravel()
unseen_performance[ens][member] = utils.evaluate_test(y_unseen, y_pred_unseen)
print(unseen_performance[ens][member])

## Create the reconstruction

In [ ]:
# Create the reconstruction and save it
# Jake calls it seen
# This should just be all SOCAT locations for all training years (not test years)
y_pred_seen = model.predict(X)

In [ ]:
# Full reconstruction 
df['pCO2_DIC_recon'] = np.nan
df.loc[unseen_sel,['pCO2_DIC_recon']] = y_pred_unseen   # Not in a SOCAT location, not even in test year
df.loc[sel,['pCO2_DIC_recon']] = y_pred_seen

In [ ]:
# All time/locations not sampled by SOCAT
df['pCO2_DIC_nosocat'] = np.nan
df.loc[unseen_sel,['pCO2_DIC_nosocat']] = y_pred_unseen
df.loc[sel,['pCO2_DIC_nosocat']] = np.nan

In [ ]:
# Only at time/locations of SOCAT sampling
df['pCO2_DIC_socat'] = np.nan
df.loc[unseen_sel,['pCO2_DIC_socat']] = np.nan
df.loc[sel,['pCO2_DIC_socat']] = y_pred_seen
     
df['pCO2_DIC'] = df['pCO2_pCO2T_diff']
             
#DS_recon = df[['net_mask','socat_mask','pCO2_DIC','pCO2_DIC_recon','pCO2_DIC_socat','pCO2_DIC_nosocat']].to_xarray()
DS_recon = df[['net_mask','combined_mask','pCO2_DIC','pCO2_DIC_recon','pCO2_DIC_socat','pCO2_DIC_nosocat']].to_xarray()

## Save reconstructions

In [ ]:
# Uncomment when actually running            
#pre_saildrone_thea.save_recon(DS_recon, recon_output_dir, approach, ens, member)   

## Save best parameters and performance metrics

In [ ]:
# Saving best parameters and performance metrics

approach_output_dir = f"{other_output_dir}/{approach}"
param_fname = f"{approach_output_dir}/{approach}_best_params_dict.pickle"
test_perform_fname = f"{approach_output_dir}/{approach}_test_performance_dict.pickle"
unseen_perform_fname = f"{approach_output_dir}/{approach}_unseen_performance_dict.pickle"

Path(approach_output_dir).mkdir(parents=True, exist_ok=True)

with open(param_fname, 'wb') as handle: #WHAT DOES wb MEAN
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(test_perform_fname, 'wb') as handle:
    pickle.dump(test_performance, handle)
with open(unseen_perform_fname, 'wb') as handle:
    pickle.dump(unseen_performance, handle)
    
# Convert performance metrics to dataframes
test_df = pd.DataFrame.from_dict({(i,j): test_performance[i][j]
                                  for i in test_performance.keys()
                                  for j in test_performance[i].keys()},
                                 orient='index')

unseen_df = pd.DataFrame.from_dict({(i,j): unseen_performance[i][j]
                                  for i in unseen_performance.keys()
                                  for j in unseen_performance[i].keys()},
                                 orient='index')

test_df.index.names = ["model","member"]
unseen_df.index.names = ["model","member"]

# Save the dataframes too
test_df_fname = f"{approach_output_dir}/{approach}_test_performance_df.pickle"
unseen_df_fname = f"{approach_output_dir}/{approach}_unseen_performance_df.pickle"

test_df.to_pickle(test_df_fname)
unseen_df.to_pickle(unseen_df_fname)    

In [ ]:
#just checking what the saved trained datafiles look like 
test_2 = pd.read_pickle("/data/artemis/workspace/theimdal/saildrone/models/trained/xg/CESM/member_016/xg_model_pC02_2D_mon_CESM_016_1x1_198201-201701.joblib")

In [ ]:
test_2

In [ ]:
#checking out what the input data for the XGB looks like
#this table was generated in script 01
df